In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

val spark = SparkSession.builder
      .appName("dsg_prg")
      .master("local[*]")
      .config("spark.sql.sources.partitionOverwriteMode","dynamic")
      .enableHiveSupport()
      .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.8:4041
SparkContext available as 'sc' (version = 2.3.1, master = local[*], app id = local-1551958444361)
SparkSession available as 'spark'


2019-03-07 17:04:09 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@5dfabe81


# CAMPAIGN DATA - 2 FILES for 6 months each

In [3]:
val campaign_6months = spark.read.parquet("../Data/Campaign_6_Months/")

campaign_6months: org.apache.spark.sql.DataFrame = [CAMPAIGN_ID: string, PLATFORM: string ... 23 more fields]


In [4]:
campaign_6months.show(5)

+-----------+--------+-------------------+--------------------+---------------+--------------------+----------+---------+-------------------+----------+---------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-----+-------+---------------+-------+------------------+-----------+---------+---------+
|CAMPAIGN_ID|PLATFORM|     EFFECTIVE_DATE|         DESCRIPTION|       CAMPAIGN|          ADVERTISER| FEED_TYPE|ATX_AD_ID|       EXPIRED_DATE|PROGRAM_ID|SOURCE_ID|   ORDER_START_DATE|     ORDER_END_DATE|           VOD_ASSET|               BRAND|CAMPAIGN_DESCRIPTION|AD_TYPE_DESCRIPTION|VODAd|VODType|goto_designator|icon_id|MASTER_DEAL_NUMBER|DEAL_NUMBER|TIME_ZONE|REGION_ID|
+-----------+--------+-------------------+--------------------+---------------+--------------------+----------+---------+-------------------+----------+---------+-------------------+-------------------+--------------------+--------------------+------

In [5]:
campaign_6months.count()

res1: Long = 1282759


In [6]:
val campaign_jan_jun = spark.read.parquet("../Data/Campaign_jan_Jun/")

campaign_jan_jun: org.apache.spark.sql.DataFrame = [CAMPAIGN_ID: string, PLATFORM: string ... 23 more fields]


In [8]:
campaign_jan_jun.show(5)

+-----------+--------+-------------------+--------------------+--------------+--------------------+----------+---------+-------------------+----------+---------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-----+-------+---------------+-------+------------------+-----------+---------+---------+
|CAMPAIGN_ID|PLATFORM|     EFFECTIVE_DATE|         DESCRIPTION|      CAMPAIGN|          ADVERTISER| FEED_TYPE|ATX_AD_ID|       EXPIRED_DATE|PROGRAM_ID|SOURCE_ID|   ORDER_START_DATE|     ORDER_END_DATE|           VOD_ASSET|               BRAND|CAMPAIGN_DESCRIPTION|AD_TYPE_DESCRIPTION|VODAd|VODType|goto_designator|icon_id|MASTER_DEAL_NUMBER|DEAL_NUMBER|TIME_ZONE|REGION_ID|
+-----------+--------+-------------------+--------------------+--------------+--------------------+----------+---------+-------------------+----------+---------+-------------------+-------------------+--------------------+--------------------+---------

In [9]:
campaign_jan_jun.count()

res4: Long = 1012620


In [11]:
val campaign = campaign_6months.union(campaign_jan_jun)

campaign: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CAMPAIGN_ID: string, PLATFORM: string ... 23 more fields]


In [12]:
campaign.count()

res5: Long = 2295379


In [13]:
campaign.distinct.count()

2019-03-07 17:07:18 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


res6: Long = 2295379


####  No Duplicates- Combine one year data to new parquet



In [15]:
campaign.repartition(1).write.parquet("../Data/Campaign/")

# LOOKUP  SOURCE

In [16]:
val source = spark.read.parquet("../Data/LOOKUP_SOURCE/")

source: org.apache.spark.sql.DataFrame = [SK_SYSTEM_ID: int, SK_PROVIDER_ID: int ... 6 more fields]


In [17]:
source.count()

res9: Long = 0


In [18]:
source.show()

+------------+--------------+---------+--------+-------------+-----------+-------+----+
|SK_SYSTEM_ID|SK_PROVIDER_ID|SOURCE_ID|SRC_NAME|SRC_LONG_NAME|SOURCE_TYPE|NETWORK|HDTV|
+------------+--------------+---------+--------+-------------+-----------+-------+----+
+------------+--------------+---------+--------+-------------+-----------+-------+----+



## This data seems similar to channel name we can use this instead ? Checked the data in tivo system there are no rows

# LOOKUP CHANNEL NAME

In [19]:
val channel = spark.read.parquet("../Data/MVSN_LOOKUP_CHANNEL_NAME/")

channel: org.apache.spark.sql.DataFrame = [SK_PROVIDER_ID: int, SOURCE_ID: int ... 5 more fields]


In [20]:
channel.count()

res11: Long = 135005


In [21]:
channel.show(5)

+--------------+---------+--------+-------------+-----------+-------+----+
|SK_PROVIDER_ID|SOURCE_ID|SRC_NAME|SRC_LONG_NAME|SOURCE_TYPE|NETWORK|HDTV|
+--------------+---------+--------+-------------+-----------+-------+----+
|             1|        0|    NULL|         null|       null|   NULL|null|
|             1|        1|    NULL|         NULL|       null|   NULL|null|
|             1|        2|    NULL|         NULL|       null|   NULL|null|
|             1|       88|    NULL|         NULL|       null|   NULL|null|
|             1|       99|    NULL|         NULL|       null|   NULL|null|
+--------------+---------+--------+-------------+-----------+-------+----+
only showing top 5 rows



# LOOKUP CAMPAIGN PROGRAM

In [22]:
val campaign_program = spark.read.parquet("../Data/CAMPAIGN_PROGRAM/")

campaign_program: org.apache.spark.sql.DataFrame = [PROGRAM_ID: int, SUBCATEGORY_ID: int ... 29 more fields]


In [23]:
campaign_program.count()

res13: Long = 757


In [24]:
campaign_program.show(5)

+----------+--------------+-----------------+----------------------+-----------+----------+---------------------+--------------------+------------+---------------+--------------+-----------------+-----------+---------+---------------+------------------+----------------+-----------------+-------+-------+------------+-----------+----------+-------------------+--------+-------------------+---------------+----------------+--------------+-----------+--------------------+
|PROGRAM_ID|SUBCATEGORY_ID|PROGRAM_SOURCE_ID|PROGRAM_AIRING_TYPE_ID|CATEGORY_ID|CAPTION_ID|PROGRAM_COLOR_TYPE_ID|        MASTER_TITLE|RELEASE_YEAR|  EPISODE_TITLE|EPISODE_NUMBER|STEREO_ENABLED_YN|STAR_RATING|SERIES_YN|RATING_ID_TV_US|RATING_ID_MOVIE_US|SERIES_MASTER_YN|PARENT_PROGRAM_ID|RUNTIME|HDTV_YN|BRACKET_TEXT|PART_NUMBER|PART_TOTAL|       ORIGINAL_ADT|SUBTITLE|         EVENT_DATE|            TAG|Cosmo_program_id|link_region_id|status_code|     update_datetime|
+----------+--------------+-----------------+-------------

# PLT POP HISTORY AND SYS POP HISTORY

In [25]:
val pltPopHistory = spark.read.parquet("../Data/mvsn_DM_PLATFORM_POPULATION_HISTORY/")

pltPopHistory: org.apache.spark.sql.DataFrame = [SK_PLATFORM_ID: int, PLATFORM: string ... 3 more fields]


In [26]:
pltPopHistory.count()

res15: Long = 17


In [30]:
pltPopHistory.show()

+--------------+--------+----------+--------+--------+
|SK_PLATFORM_ID|PLATFORM|POPULATION|  S_DATE|  E_DATE|
+--------------+--------+----------+--------+--------+
|             1|  iGuide|   6856852|20160203|20160529|
|             1|  iGuide|   6804920|20160530|20160731|
|             1|  iGuide|   6793874|20160801|20161127|
|             1|  iGuide|   6150299|20181231|20500101|
|             1|  iGuide|   6887422|20161128|20170226|
|             1|  iGuide|   6887422|20170227|20170625|
|             1|  iGuide|   6760975|20170626|20170730|
|             1|  iGuide|   6776983|20170731|20171029|
|             1|  iGuide|   6663216|20171030|20171230|
|             1|  iGuide|   6580169|20180101|20180325|
|             1|  iGuide|   6351092|20180326|20180527|
|             1|  iGuide|   6297778|20180528|20180729|
|             1|  iGuide|   6245229|20180730|20180930|
|             1|  iGuide|   6157327|20181001|20181028|
|             1|  iGuide|   6200051|20181029|20181125|
|         

In [28]:
val sysPopHistory = spark.read.parquet("../Data/mvsn_DM_SYSTEM_POPULATION_HISTORY/")

sysPopHistory: org.apache.spark.sql.DataFrame = [SK_SYSTEM_ID: int, SYSTEM: string ... 5 more fields]


In [29]:
sysPopHistory.count()

res17: Long = 24


In [31]:
sysPopHistory.show()

+------------+-----------+--------------+----------+---------------+--------+--------+
|SK_SYSTEM_ID|     SYSTEM|SK_PLATFORM_ID|POPULATION|HOUSEHOLD_RATIO|  S_DATE|  E_DATE|
+------------+-----------+--------------+----------+---------------+--------+--------+
|          40| Pittsburgh|             1|   75558.0|            2.5|20181231|20500101|
|          40| Pittsburgh|             1|  127797.0|            2.5|20170130|20170226|
|          41|Sioux Falls|             1|   97332.0|            2.5|20181231|20500101|
|          40| Pittsburgh|             1|  125605.0|            2.5|20170227|20170625|
|          40| Pittsburgh|             1|  107585.0|            2.5|20170626|20170730|
|          41|Sioux Falls|             1|  133147.0|            2.5|20170626|20170730|
|          40| Pittsburgh|             1|  105954.0|            2.5|20170731|20171029|
|          41|Sioux Falls|             1|  133147.0|            2.5|20170731|20171029|
|          40| Pittsburgh|             1|  